In [1]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/MobCommission/commissionV2/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "commissionerv2.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()
from os import path, environ
from datetime import datetime
from services.zayev.environment.market_simulator import MarketSimulator
from services.zayev.service_layer.forester import Forester
import tensorflow as tf

import numpy as np


2023-12-04 00:53:01.534207: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:


starting_time_step = datetime(year=1995, month=1, day=1, hour=10)

db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}
env_config = {
    "db_params": db_params, 
    "max_episode_steps": 2000, 
    "the_current_time_step": starting_time_step,
    "print_output": False,
    "is_test": False,
    "test_steps": 200,
    "n_step_stocks": 5,
    "n_step_cmmdties": 5,
    "preparing": True
} 

market = MarketSimulator(env_config=env_config)

In [3]:
market.reset()

(array([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]),
 array([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]),
 array([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]))

In [4]:
forest = Forester(market=market)
print("done")

done


In [13]:
rand_acts = forest.act_random()
forest.market.step(rand_acts)

test output: indx, 2 chng > offer_vol 2761.38362784536> 2761.3836278453596


((array([[-0.02846564, -0.03530218, -0.04147502, -0.02905806, -0.01986838],
         [ 0.00430969,  0.01183701,  0.02041852,  0.02235196,  0.00889054],
         [ 0.02799265,  0.06022457,  0.0617041 ,  0.03877704,  0.04074955],
         [-0.02152703, -0.01077146,  0.02075958,  0.02063911,  0.00219324],
         [-0.01430887, -0.00234406, -0.00568703, -0.00963894, -0.00016054]]),
  array([[-0.05936728, -0.06399051, -0.02248568, -0.0189497 , -0.04376922],
         [ 0.01214107, -0.00165588, -0.00261344,  0.00625529,  0.01128686],
         [-0.02308626, -0.03035286, -0.01251526,  0.00521165, -0.01703313],
         [-0.01478374, -0.01294796,  0.00352659,  0.00987303, -0.01082911],
         [-0.01757247, -0.01522848, -0.01103836, -0.01568249, -0.01434613],
         [-0.03790692, -0.03867508, -0.02344572, -0.02418792, -0.02557386]]),
  array([[0.00198781, 0.00067794, 0.00310005, 0.01854009, 0.00094466],
         [0.00557802, 0.00588305, 0.00050826, 0.00020597, 0.00189556],
         [0.003439